<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тест" data-toc-modified-id="Тест-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тест</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для интернет-магазина

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

In [1]:
!pip install pymystem3
from pymystem3 import Mystem

In [2]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
import spacy
import pandas as pd
import os
import numpy as np 
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier 
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
import time
import warnings
warnings.filterwarnings('ignore')
import re 
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

     ---------------------------------------- 12.8/12.8 MB 8.0 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-07-11 21:29:06.852275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-11 21:29:06.852296: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\taras\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\taras\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
pth1 = 'toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.isna().sum()

text     0
toxic    0
dtype: int64

In [7]:
s = data['toxic'].value_counts()
s[1]/s[0]

0.1131876717871444

Проведем чистку, токенизацию и лемматизацию с помощью WordNetLemmatizer

In [8]:
corp = data['text'].values

In [9]:
corpus = []
s = r'[^a-zA-Z0-9]'
for i in corp:
    cleared = re.sub(s, " ", i)
    corpus.append(" ". join(cleared.split()))

In [10]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [11]:
def lemmatizered(corpus):
    corpus_lem = []
    for i in corpus:
        s = nlp(i)
        corpus_lem.append(" ".join([token.lemma_ for token in s]))
    return corpus_lem

In [12]:
data['lemm_text']  = lemmatizered(corpus)

In [13]:
sample_size = 50000
corpus = data.sample(n=sample_size,random_state=2007).reset_index(drop=True)
print('соотношение классов в датасете corpus\n', corpus.toxic.value_counts()/corpus.shape[0]*100)

соотношение классов в датасете corpus
 0    89.736
1    10.264
Name: toxic, dtype: float64


In [14]:
# wnl = WordNetLemmatizer()
# def lemmatizered(corpus):
#     corpus_lem = []
#     for i in corpus:
#         s = nltk.word_tokenize(i)
#         corpus_lem.append(' '.join([wnl.lemmatize(k) for k in s]))
#     return corpus_lem

In [15]:
# data['lemm_text']  = lemmatizered(corpus)

In [16]:
corpus.shape

(50000, 3)

В сете 2 столбца: toxic, text. Из них 90% нетоксичных комментариев

## Подготовка

In [17]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\taras\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
text_features = ['lemm_text']
target = 'toxic'
features = 'lemm_text'

In [19]:
train_corpus, test_corpus = train_test_split(corpus, test_size = 0.2, random_state=2007,stratify = corpus['toxic'])

In [20]:
train_corpus

,text,toxic,lemm_text
2753,(It's true. You wikipedia kids need to get off...,1,it s true you wikipedia kid need to get off th...
10360,If your referring to the comments on my talk p...,0,if your refer to the comment on my talk page I...
31689,"""\nHello - I have no intention whatsoever of ...",0,hello I have no intention whatsoever of make W...
49618,way.\n\nBut I think it's time to point out tha...,0,way but I think it s time to point out that th...
22986,"Hey daniel, this seems to be a large problem a...",0,hey daniel this seem to be a large problem at ...
...,...,...,...
17767,Don't know what happened and not sure how to u...,0,Don t know what happen and not sure how to use...
6043,"Text is not small, it will take some time to r...",0,text be not small it will take some time to re...
26300,"Section titled ==Journeyperson== \n\nActually,...",0,section title Journeyperson Actually it be jus...
13492,A message from Jasonceyre \n\n f uc k you ho...,1,a message from Jasonceyre f uc k you hoe you a...


In [21]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

In [22]:
features_train = count_tf_idf.fit_transform(train_corpus['lemm_text'])
features_test = count_tf_idf.transform(test_corpus['lemm_text'])

In [23]:
target_train = train_corpus['toxic']
target_test = test_corpus['toxic']

In [24]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(40000, 72354)
(10000, 72354)
(40000,)
(10000,)


Подготовили данные, выделили фичи и таргеты, после лемматизации и чистки.

## Обучение

LogisticRegression

In [25]:
LR = LogisticRegression(class_weight='balanced', 
                        random_state=2007,
                        n_jobs=-1,
                        solver='liblinear'
                       )

In [26]:
LR_params = {"max_iter":[10,50,10],
             'C': [0.1, 1, 10],
            }

In [27]:
LR_gsearch = GridSearchCV(LR, LR_params, scoring='f1', cv=5)
LR_gsearch.fit(features_train, target_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced', n_jobs=-1,
                                          random_state=2007,
                                          solver='liblinear'),
             param_grid={'C': [0.1, 1, 10], 'max_iter': [10, 50, 10]},
             scoring='f1')

In [28]:
LR_gsearch.best_params_

{'C': 10, 'max_iter': 10}

In [29]:
LR_gsearch.best_score_

0.754341463307493

CatBoost - очень долгая, можно попробовать уменьшить выборку в n раз

In [30]:
# %%time
# CB_model = CatBoostClassifier(random_state = 2007)
# CB_param_search = { 
#                     'learning_rate': [0.03, 0.1],
#                     'depth': [4, 6, 10]
# }
# CB_gsearch = GridSearchCV(n_jobs = -1, estimator=CB_model, cv=3, param_grid=CB_param_search, scoring = 'f1')

# CB_gsearch.fit(features_train, target_train, verbose=None)

In [31]:
# CB_gsearch.best_params_

In [32]:
CB = CatBoostClassifier( 
    text_features=text_features,
    verbose=100,
    loss_function='Logloss',
    eval_metric='F1',
    task_type="GPU",
    iterations=1000,
    learning_rate=0.2,
    random_seed = 2007,
    auto_class_weights = 'Balanced',
    text_processing = {
        "tokenizers" : [{
            "tokenizer_id" : "Space",
            "separator_type" : "ByDelimiter",
            "delimiter" : " "
        }],

        "dictionaries" : [{
            "dictionary_id" : "BiGram",
            "token_level_type": "Letter",
            "max_dictionary_size" : "150000",
            "occurrence_lower_bound" : "1",
            "gram_order" : "2"
        },{
            "dictionary_id" : "Trigram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "3"
        }],

        "feature_processing" : {
            "default" : [
                    {
                    "dictionaries_names" : ["BiGram", "Trigram"],
                    "feature_calcers" : ["BoW"],
                    "tokenizers_names" : ["Space"]
                },
                    {
                "dictionaries_names" : ["BiGram", "Trigram"],
                "feature_calcers" : ["NaiveBayes"],
                "tokenizers_names" : ["Space"]
            },{
                "dictionaries_names" : [ "BiGram", "Trigram"],
                "feature_calcers" : ["BM25"],
                "tokenizers_names" : ["Space"]
            },
            ],
        }
    }
)

In [33]:
CB.fit(
    train_corpus[text_features], train_corpus[target],
    eval_set=(test_corpus[text_features], test_corpus[target]),
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8019772	test: 0.8122097	best: 0.8122097 (0)	total: 46.2ms	remaining: 46.2s
100:	learn: 0.8967984	test: 0.8653464	best: 0.8653464 (100)	total: 3.9s	remaining: 34.7s
200:	learn: 0.9210980	test: 0.8637332	best: 0.8723728 (135)	total: 7.74s	remaining: 30.8s
300:	learn: 0.9336392	test: 0.8655891	best: 0.8723728 (135)	total: 11.6s	remaining: 26.9s
400:	learn: 0.9413552	test: 0.8652908	best: 0.8723728 (135)	total: 15.5s	remaining: 23.1s
500:	learn: 0.9489553	test: 0.8647845	best: 0.8723728 (135)	total: 19.4s	remaining: 19.3s
600:	learn: 0.9561321	test: 0.8643505	best: 0.8723728 (135)	total: 23.5s	remaining: 15.6s
700:	learn: 0.9617361	test: 0.8573306	best: 0.8723728 (135)	total: 27.4s	remaining: 11.7s
800:	learn: 0.9690470	test: 0.8504691	best: 0.8723728 (135)	total: 31.4s	remaining: 7.8s
900:	learn: 0.9724066	test: 0.8504365	best: 0.8723728 (135)	total: 35.2s	remaining: 3.87s
999:	learn: 0.9770365	test: 0.8466743	best: 0.8723728 (135)	total: 39.1s	remaining: 0us
bestTest = 0.8723

In [34]:
train_full = corpus.sample(frac=0.8, random_state=2007).copy()
test = corpus[~corpus.index.isin(train_full.index)].copy()

In [35]:
train_full_tf = train_full[['lemm_text','toxic']].copy()

In [36]:
CB2 = CatBoostClassifier(verbose = 100,
                        learning_rate=0.7,
                        early_stopping_rounds=200,
                        eval_metric='AUC')

In [37]:
CB2.fit(train_full_tf[['lemm_text']], train_full_tf[['toxic']],
       eval_set=(test[['lemm_text']], test[['toxic']]),
       text_features=['lemm_text'])

0:	test: 0.8126666	best: 0.8126666 (0)	total: 38.9ms	remaining: 38.8s
100:	test: 0.9582289	best: 0.9587021 (74)	total: 5.6s	remaining: 49.8s
200:	test: 0.9548237	best: 0.9587021 (74)	total: 11.2s	remaining: 44.5s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9587021343
bestIteration = 74

Shrink model to first 75 iterations.


Forest

In [38]:
Forest = RandomForestClassifier(class_weight='balanced', n_jobs=-1 )
forest_params = { 'n_estimators': [100],
                    'max_depth' : [i for i in range(13,15)]
                }
forest_gsearch = GridSearchCV(Forest, forest_params, scoring='f1', cv=5)
forest_gsearch.fit(features_train, target_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              n_jobs=-1),
             param_grid={'max_depth': [13, 14], 'n_estimators': [100]},
             scoring='f1')

In [39]:
forest_gsearch.best_params_

{'max_depth': 13, 'n_estimators': 100}

In [40]:
forest_gsearch.best_score_

0.38713036182376187

LGBMClassifier

In [41]:
LGBM_model = LGBMClassifier(random_state = 2007)
LGBM_params = {
  'n_estimators': [200],
  'learning_rate': [0.01, 0.1],
  'max_depth': [i for i in range(8,15)]}

LGBM_gsearch = GridSearchCV(n_jobs = -1, estimator=LGBM_model, cv=5, param_grid=LGBM_params, scoring = 'f1')

LGBM_gsearch.fit(features_train, target_train)


GridSearchCV(cv=5, estimator=LGBMClassifier(random_state=2007), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1],
                         'max_depth': [8, 9, 10, 11, 12, 13, 14],
                         'n_estimators': [200]},
             scoring='f1')

In [42]:
LGBM_gsearch.best_params_

{'learning_rate': 0.1, 'max_depth': 14, 'n_estimators': 200}

In [43]:
LGBM_gsearch.best_score_

0.7221131141460556

CatBoost 2 модели с параметрами и без

## Тест

LogisticRegression

In [44]:
LR_test = LogisticRegression(class_weight='balanced',
                             C = 10,
                        random_state=2007,
                        n_jobs=-1,
                        solver='liblinear',
                        max_iter = 10
                       )
LR_test.fit(features_train, target_train)
LR_pred = LR_test.predict(features_test)
LR_F1 = f1_score(target_test, LR_pred)
LR_F1

0.7580645161290323

Forest

In [45]:
Forest_test = RandomForestClassifier(class_weight='balanced',
                                 random_state=2007,
                                 n_jobs=-1,
                                 max_depth =14,
                                 n_estimators = 100
                       )
Forest_test.fit(features_train, target_train)
Forest_pred = Forest_test.predict(features_test)
Forest_F1 = f1_score(target_test, Forest_pred)
Forest_F1

0.3967916961547535

LGBMClassifier

In [46]:
LGBM_test = LGBMClassifier(class_weight='balanced',
                           random_state=2007,
                                   n_jobs=-1,
                                   n_estimators = 200,
                                   learning_rate=0.01,
                                   max_depth=4,
                                   )
LGBM_test.fit(features_train, target_train)
LGBM_pred = LGBM_test.predict(features_test)
LGBM_F1 = f1_score(target_test, LGBM_pred)
LGBM_F1

0.6427703523693803

In [47]:
CBT = f1_score(CB.predict(test_corpus[text_features]),test_corpus[target])
CBT

0.657471264367816

In [48]:
CBT2 = f1_score(CB2.predict(test[['lemm_text']]),test[['toxic']])
CBT2

0.7618522601984565

## Выводы

Лучшей моделью оказалась LogisticRegression, CatBoost только они удовлетворяет порогу в 0.75 f1 метрики.
